# Vladimir Nikiforov

### PART 0 - PREPARE ENV

In [174]:
# Stop previously runned server
!docker stop ht_pg_server
!docker rm $(docker ps -a -q)
!docker rmi $(docker images -q)

ht_pg_server
620c864dc1b5
Untagged: postgres:11
Untagged: postgres@sha256:68b49a280d2fbe9330c0031970ebb72015e1272dfa25f0ed7557514f9e5ad7b7
Deleted: sha256:53912975086f1470f877922e3eb79c4f17c403fcb0b51b1f45df9b623987600e
Deleted: sha256:859135cc89a4161becb56082dc95772d26d157bc67622be28ccab159cee9c520
Deleted: sha256:85d5ba072f703aed8ed136ee24ef0a5c22fb818e307db40be0637df1691546fe
Deleted: sha256:44bc37a16317acf120ebcb0b8aa2cf8738f0dcc40ca47f18d9d5789099089cd0
Deleted: sha256:050e2fbc4c511fff109624929dcc24d61f4565052ffd6e30e3d46d42ce65c576
Deleted: sha256:6e7901ce413d382c6544cf07b5370341507a4734525947ecef28944c041e39b6
Deleted: sha256:c0d5a0e28851bc9a301c0cb92e03c036c1a4b0d8da1efc4a16a921abdd641e3e
Deleted: sha256:21cf04dd5b4e7ddbe4a7f122b532038c351eef9057ff71940a1a0a36069fabd5
Deleted: sha256:5eba02d7eb8edfd2d0c4503cd6976cfb526e96c4f62960bac5efdf86a1e71e4f
Deleted: sha256:0e0ff67afa7921baa6b3c7afd83664752663b5b8704c5236941359003e6f8887
Deleted: sha256:915e9e2c13848dc5933cea4c23b53649971

In [175]:
# Run the server:
!docker run -d --name ht_pg_server -v ht_dbdata:/var/lib/postgresql/data -p 54320:5432 postgres:11

Unable to find image 'postgres:11' locally
11: Pulling from library/postgres














Digest: sha256:68b49a280d2fbe9330c0031970ebb72015e1272dfa25f0ed7557514f9e5ad7b7
Status: Downloaded newer image for postgres:11
417598d11fbee5a7da5a037dcd217abc78c507bcefe307fe729ae10d2f3a9aaf


In [176]:
# Checking the logs to see if it is running:
!docker logs ht_pg_server

2019-08-03 08:01:15.410 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2019-08-03 08:01:15.410 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2019-08-03 08:01:15.421 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2019-08-03 08:01:15.451 UTC [26] LOG:  database system was shut down at 2019-08-03 08:00:51 UTC
2019-08-03 08:01:15.467 UTC [1] LOG:  database system is ready to accept connections


In [177]:
# Creating the database:
!docker exec -it ht_pg_server psql -U postgres -c "drop database ht_db"
!docker exec -it ht_pg_server psql -U postgres -c "create database ht_db"

DROP DATABASE
CREATE DATABASE


In [178]:
!cat ../code/etl.py

import psycopg2 as pg
import yaml
from pathlib import Path
import os
import pandas as pd

def create_tables(config, connection):
    cur = connection.cursor()
    for table in config:
        name = table.get('name')
        schema = table.get('schema')
        ddl = f"""CREATE TABLE IF NOT EXISTS {name} ({schema})"""
        cur.execute(ddl)

    connection.commit()
    cur.close()

def transform_tables(config):

    data_path = "../data/"

    for table in config:
        table_name = table.get('name')
        table_source = os.path.join(data_path,f"{table_name}.csv")
        table_cols = []
        for i in table.get('columns'):
            table_cols.append(str.upper(i))
        df = pd.read_csv(table_source)
        df_reorder = df[table_cols]  # rearrange column here
        df_reorder.to_csv(table_source, index=False)

def load_tables(config, connection):

    # iterate and load
    cur = connection.cursor()
    data_path = "../data/"

    for table in config:
        table_name

In [179]:
!python ../code/etl.py

/home/truename/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [180]:
!cat ../code/features.py

# create features and store them in the DB

import psycopg2 as pg
import yaml
from pathlib import Path
import os
import pandas as pd
from datetime import datetime
import dateutil


def prepare_user_features(users_file_name):
    user_df = pd.read_csv(users_file_name)
    # There are NaN in TERMS_VERSION - fill it by "Undefined" version
    user_df['TERMS_VERSION'] = user_df['TERMS_VERSION'].fillna("Undefined")
    user_df.drop('STATE', axis=1, inplace=True)
    # Create AGE variable as age on CREATED_DATE date of profile in system
    user_df['AGE'] = user_df.apply(lambda x: datetime.strptime(x['CREATED_DATE'], '%Y-%m-%d %H:%M:%S.%f').year-x['BIRTH_YEAR'], axis=1)
    # We don't need BIRTH_YEAR anymore - drop it
    user_df.drop('BIRTH_YEAR', axis=1, inplace=True)
    return user_df


def prepare_transactions_features(user_df, transactions_file_name):
    trn_df = pd.read_csv(transactions_file_name)
    df = trn_df.drop('ID', axis=1).merge(user_df, left_on="USER_ID", right_on="ID").dro

In [181]:
!python ../code/features.py

/home/truename/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
../code/features.py:64: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if old_col in dataset['DAY_OF_TRANSACTION'].unique()
../code/features.py:81: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if old_col in dataset['HOUR_OF_TRANSACTION'].unique()
